In [1]:
import numpy as np
import keras.backend as K
from keras.regularizers import l2
import keras
from keras.models import Model, load_model
from keras.layers import Input, LSTM, Dense, GRU, Embedding, Bidirectional, Flatten, Dropout, Lambda
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import one_hot
from keras.utils import to_categorical
from keras.callbacks import History ,ModelCheckpoint, EarlyStopping
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import add, dot, concatenate, multiply, average
import numpy as np
from keras.layers import Dot
from keras.initializers import Zeros
import random
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
from keras.engine.topology import Layer
from sklearn.preprocessing import scale
from sklearn.utils import shuffle
from keras.utils import get_custom_objects
from keras.preprocessing.sequence import pad_sequences
from keras.utils import plot_model
from keras.utils.vis_utils import model_to_dot
from IPython.display import SVG
%env CUDA_VISIBLE_DEVICES=3
import pickle

Using TensorFlow backend.


env: CUDA_VISIBLE_DEVICES=3


In [2]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

print("train data shape",train_data.shape)
print("test data shape", test_data.shape)

all_UMpair = pd.concat([train_data,test_data])[["UserID","MovieID"]]

# create dictionary to encode UserID and MovieID
user2id = {}
movie2id = {}
for i, UserID in enumerate(all_UMpair["UserID"].unique()):
    user2id[UserID] = i
for j, MovieID in enumerate(all_UMpair["MovieID"].unique()):
    movie2id[MovieID] = j
num_user = len(user2id)
num_movie = len(movie2id)
print("number of user",num_user)
print("number of movie",num_movie)

train data shape (899873, 4)
test data shape (100336, 3)
number of user 6040
number of movie 3706


In [3]:
# implicit feed back

all_UMpair["UserID"] = all_UMpair["UserID"].apply(lambda x:user2id[x])
all_UMpair["MovieID"] = all_UMpair["MovieID"].apply(lambda x:movie2id[x])
def get_feedback(X, num_users, num_movies):
    feedback_u = [[] for u in range(num_users)]
    feedback_m = [[] for i in range(num_movies)]

    for u, m in zip(X[:, 0], X[:, 1]):
        feedback_u[u].append(m+1)
        feedback_m[m].append(u+1)

    return feedback_u, feedback_m
feedback_user, feedback_movie = get_feedback(all_UMpair.values, num_user,num_movie)
feedback_user, feedback_movie = pad_sequences(feedback_user), pad_sequences(feedback_movie)
feedback_user_embedding = Embedding(num_user, feedback_user.shape[1], 
                                    trainable = False, weights = [feedback_user])
feedback_movie_embedding = Embedding(num_movie, feedback_movie.shape[1], 
                                     trainable = False, weights = [feedback_movie])
print("feedback user",feedback_user.shape)
print("feedback movie",feedback_movie.shape)

feedback user (6040, 2314)
feedback movie (3706, 3428)


In [4]:
# dealing with training data
train_user = np.array([ user2id[i] for i in train_data["UserID"]])
train_movie = np.array([ movie2id[j] for j in train_data["MovieID"]])
train_rating = np.array(train_data["Rating"])

total_sample_size =  len(train_rating)
print("total train sample",total_sample_size)

train_user, train_movie, train_rating = shuffle(train_user, train_movie, train_rating)
mean = np.mean(train_rating)
std = np.std(train_rating)
normal_rating = (train_rating - mean)/std
print(normal_rating)

# normal_rating = scale(train_rating)
print("std",std)
print("mean",mean)
# print(train_rating)
# print(normal_rating)

total train sample 899873
[ 1.2698459  -1.41616564  0.37450872 ...,  0.37450872 -0.52082846
 -0.52082846]
std 1.11689766115
mean 3.58171208604


In [5]:
# load other feature
movie_df = pd.read_csv("movies.csv",sep="::")
movie_df["year"] = movie_df["Title"].apply(lambda x:str(x)[-5:-1])
# encode movieID
movie_year = np.zeros(num_movie)
for ID, year in zip(movie_df["movieID"], movie_df["year"]):
    try:
        movie_year[movie2id[ID]] = year
    except:
        pass
enc = OneHotEncoder()
movie_year = enc.fit_transform(movie_year.reshape(-1,1)).toarray()
movie_year_emb = Embedding(num_movie, movie_year.shape[1], trainable = False, weights = [movie_year],
                          name='movie_year_embedding')

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [6]:
movie_year[1]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
        0.,  0.,  0.])

In [7]:
len(np.unique(movie_year))

2

In [9]:

class WeightedAvgOverTime(Layer):
    """
    The code of this layer is from
        https://github.com/WindQAQ/ML2017/blob/master/hw6/train.py
    """
    def __init__(self, **kwargs):
        self.supports_masking = True
        super(WeightedAvgOverTime, self).__init__(**kwargs)
    def call(self, x, mask=None):
        if mask is not None:
            mask = K.cast(mask, K.floatx())
            mask = K.expand_dims(mask, axis=-1)
            s = K.sum(mask, axis=1)
            if K.equal(s, K.zeros_like(s)) is None:
                return K.mean(x, axis=1)
            else:
                return K.cast(K.sum(x * mask, axis=1) / K.sqrt(s), K.floatx())
        else:
            return K.mean(x, axis=1)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])

    def compute_mask(self, x, mask=None):
        return None

    def get_config(self):
        base_config = super(WeightedAvgOverTime, self).get_config()
        return dict(list(base_config.items()))
    
def rmse(y_true, y_pred):
    y_pred = K.clip(y_pred, 1.0, 5.0)
    return K.sqrt(K.mean(K.pow(y_true - y_pred, 2)))

get_custom_objects().update({"rmse": rmse})

epochs = 1000
lant_dim = 15
lmbda = 0.00001
batch_size = 256
for i in range(15,20):
    train_user, train_movie, train_rating = shuffle(train_user, train_movie, train_rating)
    ###### Two channels input #####
    user_input = Input(shape=(1,), name='user')
    movie_input = Input(shape=(1,), name='movie')
    # Embedding
    user_embedding = Embedding(input_dim=num_user, 
                                  output_dim=lant_dim,
                                  embeddings_regularizer=l2(lmbda))(user_input)

    movie_embedding = Embedding(input_dim=num_movie, 
                               output_dim=lant_dim,
                               embeddings_regularizer=l2(lmbda))(movie_input)
    # Flatten
    user_flat = Flatten()(user_embedding)
    movie_flat = Flatten()(movie_embedding)
    
    # Dropout
    user_dropout = Dropout(0.1)(user_flat)
    movie_dropout = Dropout(0.1)(movie_flat)
    
    # implicit feedback
    feedback_user_channel = Flatten()(feedback_user_embedding(user_input))
    feedback_user_channel = Embedding(num_movie+1, lant_dim, embeddings_initializer=Zeros(),
                                     embeddings_regularizer=l2(lmbda),
                                      mask_zero = True)(feedback_user_channel)
    feedback_user_channel = Dropout(0.1)(feedback_user_channel)
    feedback_user_channel = WeightedAvgOverTime(name="weight_average")(feedback_user_channel)
    
    feedback_movie_channel = Flatten()(feedback_movie_embedding(movie_input))
    feedback_movie_channel = Embedding(num_user+1, lant_dim, embeddings_initializer=Zeros(),
                                     embeddings_regularizer=l2(lmbda),
                                      mask_zero = True)(feedback_movie_channel)
    feedback_movie_channel = Dropout(0.1)(feedback_movie_channel)
    print(feedback_movie_channel.shape)
    feedback_movie_channel = WeightedAvgOverTime()(feedback_movie_channel)
    print(feedback_movie_channel.shape)
    # merge 
    user_channel = add([user_dropout,feedback_user_channel])
    movie_channel = add([movie_dropout, feedback_movie_channel])
#     print(user_channel.shape)
#     print(movie_channel.shape)
    
    dot_layer = dot(inputs=[user_channel, movie_channel], axes = -1)
    #print(dot_layer.shape)
    # add bias
    user_bias = Embedding(input_dim = num_user, output_dim=1,
                          embeddings_regularizer=l2(lmbda), 
                             name='user_bias')(user_input)

    movie_bias = Embedding(input_dim = num_movie, output_dim=1,
                           embeddings_regularizer=l2(lmbda), 
                          name='movie_bias')(movie_input)
    
    user_bias_flat = Flatten()(user_bias)
    movie_bias_flat = Flatten()(movie_bias)
    # movie year 
    movie_year_channel = Flatten()(movie_year_emb(movie_input))
    print(movie_year_channel.shape)
    movie_year_dense = Dense(1, activation="linear")(movie_year_channel)
    predicted_preference = add(inputs=[dot_layer, user_bias_flat, movie_bias_flat,movie_year_dense])
    predicted_preference = Lambda(lambda x: x + K.constant(3.5817, dtype=K.floatx()))(predicted_preference)
#     predicted_preference = dot_layer
    model = Model(inputs=[user_input, movie_input], outputs=predicted_preference)
    model.compile(loss='mse', optimizer='adam', metrics=[rmse])
    model.summary()
    hist = History()
    check_save  = ModelCheckpoint("models/model_"+str(i)+".h5",monitor='val_rmse',save_best_only=True)
    early_stop = EarlyStopping(monitor="val_rmse", patience=3)
    model.fit([train_user, train_movie], train_rating,
              batch_size=batch_size,
              epochs=epochs,
              validation_split=0.1,
             callbacks=[ hist, early_stop,check_save])
    print(np.min(hist.history["val_rmse"]))

(?, ?, 15)
(?, 15)
(?, ?)
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
user (InputLayer)                (None, 1)             0                                            
____________________________________________________________________________________________________
movie (InputLayer)               (None, 1)             0                                            
____________________________________________________________________________________________________
embedding_1 (Embedding)          (None, 1, 2314)       13976560    user[0][0]                       
____________________________________________________________________________________________________
embedding_2 (Embedding)          (None, 1, 3428)       12704168    movie[0][0]                      
_________________________________________________________________

809885/809885 [==============================] - 83s - loss: 0.6618 - rmse: 0.7892 - val_loss: 0.7326 - val_rmse: 0.8327
Epoch 9/1000
809885/809885 [==============================] - 84s - loss: 0.6596 - rmse: 0.7868 - val_loss: 0.7359 - val_rmse: 0.8339
Epoch 10/1000
809885/809885 [==============================] - 84s - loss: 0.6578 - rmse: 0.7848 - val_loss: 0.7366 - val_rmse: 0.8337
0.831191171208
(?, ?, 15)
(?, 15)
(?, ?)
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
user (InputLayer)                (None, 1)             0                                            
____________________________________________________________________________________________________
movie (InputLayer)               (None, 1)             0                                            
________________________________________________________________

809885/809885 [==============================] - 84s - loss: 0.6761 - rmse: 0.8032 - val_loss: 0.7301 - val_rmse: 0.8357
Epoch 6/1000
809885/809885 [==============================] - 84s - loss: 0.6694 - rmse: 0.7970 - val_loss: 0.7338 - val_rmse: 0.8360
Epoch 7/1000
809885/809885 [==============================] - 84s - loss: 0.6647 - rmse: 0.7924 - val_loss: 0.7365 - val_rmse: 0.8365
Epoch 8/1000
809885/809885 [==============================] - 83s - loss: 0.6617 - rmse: 0.7892 - val_loss: 0.7396 - val_rmse: 0.8373
Epoch 9/1000
809885/809885 [==============================] - 83s - loss: 0.6592 - rmse: 0.7866 - val_loss: 0.7408 - val_rmse: 0.8368
0.835694474342
(?, ?, 15)
(?, 15)
(?, ?)
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
user (InputLayer)                (None, 1)             0                                            

809885/809885 [==============================] - 83s - loss: 0.7039 - rmse: 0.8256 - val_loss: 0.7220 - val_rmse: 0.8353
Epoch 4/1000
809885/809885 [==============================] - 84s - loss: 0.6878 - rmse: 0.8129 - val_loss: 0.7222 - val_rmse: 0.8331
Epoch 5/1000
809885/809885 [==============================] - 83s - loss: 0.6777 - rmse: 0.8042 - val_loss: 0.7216 - val_rmse: 0.8304
Epoch 6/1000
809885/809885 [==============================] - 83s - loss: 0.6704 - rmse: 0.7977 - val_loss: 0.7280 - val_rmse: 0.8326
Epoch 7/1000
809885/809885 [==============================] - 83s - loss: 0.6659 - rmse: 0.7932 - val_loss: 0.7287 - val_rmse: 0.8316
Epoch 8/1000
809885/809885 [==============================] - 83s - loss: 0.6627 - rmse: 0.7899 - val_loss: 0.7323 - val_rmse: 0.8327
Epoch 9/1000
809885/809885 [==============================] - 83s - loss: 0.6600 - rmse: 0.7870 - val_loss: 0.7319 - val_rmse: 0.8311
0.830449929157
(?, ?, 15)
(?, 15)
(?, ?)
__________________________________

809885/809885 [==============================] - 84s - loss: 0.8131 - rmse: 0.8965 - val_loss: 0.7550 - val_rmse: 0.8617
Epoch 2/1000
809885/809885 [==============================] - 84s - loss: 0.7310 - rmse: 0.8455 - val_loss: 0.7350 - val_rmse: 0.8464
Epoch 3/1000
809885/809885 [==============================] - 84s - loss: 0.7031 - rmse: 0.8251 - val_loss: 0.7263 - val_rmse: 0.8381
Epoch 4/1000
809885/809885 [==============================] - 84s - loss: 0.6874 - rmse: 0.8127 - val_loss: 0.7258 - val_rmse: 0.8353
Epoch 5/1000
809885/809885 [==============================] - 84s - loss: 0.6771 - rmse: 0.8038 - val_loss: 0.7289 - val_rmse: 0.8352
Epoch 6/1000
809885/809885 [==============================] - 84s - loss: 0.6704 - rmse: 0.7976 - val_loss: 0.7303 - val_rmse: 0.8338
Epoch 7/1000
809885/809885 [==============================] - 84s - loss: 0.6659 - rmse: 0.7933 - val_loss: 0.7325 - val_rmse: 0.8340
Epoch 8/1000
809885/809885 [==============================] - 84s - loss: 0

Train on 809885 samples, validate on 89988 samples
Epoch 1/1000
809885/809885 [==============================] - 85s - loss: 0.8130 - rmse: 0.8963 - val_loss: 0.7526 - val_rmse: 0.8605
Epoch 2/1000
809885/809885 [==============================] - 85s - loss: 0.7319 - rmse: 0.8461 - val_loss: 0.7331 - val_rmse: 0.8449
Epoch 3/1000
809885/809885 [==============================] - 84s - loss: 0.7031 - rmse: 0.8253 - val_loss: 0.7265 - val_rmse: 0.8383
Epoch 4/1000
809885/809885 [==============================] - 85s - loss: 0.6876 - rmse: 0.8128 - val_loss: 0.7242 - val_rmse: 0.8341
Epoch 5/1000
809885/809885 [==============================] - 84s - loss: 0.6766 - rmse: 0.8036 - val_loss: 0.7267 - val_rmse: 0.8334
Epoch 6/1000
809885/809885 [==============================] - 84s - loss: 0.6698 - rmse: 0.7973 - val_loss: 0.7296 - val_rmse: 0.8336
Epoch 7/1000
809885/809885 [==============================] - 84s - loss: 0.6647 - rmse: 0.7924 - val_loss: 0.7316 - val_rmse: 0.8336
Epoch 8/100

In [11]:
test_user = np.array([user2id[i] for i in test_data["UserID"]])
test_movie = np.array([movie2id[i] for i in test_data["MovieID"]])

In [12]:
pred_rating = model.predict([test_user,test_movie])
print(pred_rating)

[[ 4.64944649]
 [ 4.11228228]
 [ 4.28679085]
 ..., 
 [ 1.10693383]
 [ 3.04244924]
 [ 3.92776585]]


In [13]:
# load model 
import timeit

start = timeit.default_timer()
get_custom_objects().update({"rmse": rmse,"mean":mean,"WeightedAvgOverTime":WeightedAvgOverTime})
model_1 = load_model("models/model_0.h5")
model_2 = load_model("models/model_1.h5")
model_3 = load_model("models/model_2.h5")
model_4 = load_model("models/model_3.h5")
model_5 = load_model("models/model_4.h5")
model_6 = load_model("models/model_5.h5")
model_7 = load_model("models/model_6.h5")
model_8 = load_model("models/model_7.h5")
model_9 = load_model("models/model_8.h5")
model_10 = load_model("models/model_9.h5")
model_11 = load_model("models/model_10.h5")
model_12 = load_model("models/model_11.h5")
model_13 = load_model("models/model_12.h5")
model_14 = load_model("models/model_13.h5")
model_15 = load_model("models/model_14.h5")
model_16 = load_model("models/model_15.h5")
model_17 = load_model("models/model_16.h5")
model_18 = load_model("models/model_17.h5")
model_19 = load_model("models/model_18.h5")
model_20 = load_model("models/model_19.h5")
pred_rating1 = model_1.predict([test_user,test_movie])
pred_rating2 = model_2.predict([test_user,test_movie])
pred_rating3 = model_3.predict([test_user,test_movie])
pred_rating4 = model_4.predict([test_user,test_movie])
pred_rating5 = model_5.predict([test_user,test_movie])
pred_rating6 = model_6.predict([test_user,test_movie])
pred_rating7 = model_7.predict([test_user,test_movie])
pred_rating8 = model_8.predict([test_user,test_movie])
pred_rating9 = model_9.predict([test_user,test_movie])
pred_rating10 = model_10.predict([test_user,test_movie])
pred_rating11 = model_11.predict([test_user,test_movie])
pred_rating12 = model_12.predict([test_user,test_movie])
pred_rating13 = model_13.predict([test_user,test_movie])
pred_rating14 = model_14.predict([test_user,test_movie])
pred_rating15 = model_15.predict([test_user,test_movie])
pred_rating16 = model_16.predict([test_user,test_movie])
pred_rating17 = model_17.predict([test_user,test_movie])
pred_rating18 = model_18.predict([test_user,test_movie])
pred_rating19 = model_19.predict([test_user,test_movie])
pred_rating20 = model_20.predict([test_user,test_movie])
stop = timeit.default_timer()

print (stop - start)

282.2660226370208


In [14]:
pred_rating = np.average(np.concatenate((pred_rating1,pred_rating2,pred_rating3,pred_rating4,
                          pred_rating5,pred_rating6,pred_rating7,pred_rating8,pred_rating9,
                          pred_rating10,pred_rating11,pred_rating12,pred_rating13,pred_rating14,
                                        pred_rating15,pred_rating16,pred_rating17,pred_rating18,
                                        pred_rating19,pred_rating20),axis=1),axis=1)
# pred_rating = np.average(np.concatenate((pred_rating1,pred_rating2,pred_rating3,pred_rating4,
#                           pred_rating5,pred_rating6,pred_rating7),axis=1),axis=1)

In [15]:
pred_rating

array([ 4.44880772,  4.11352539,  4.17152166, ...,  1.38329113,
        3.13445687,  3.92536783], dtype=float32)

In [16]:
sampleSubmission = pd.read_csv("SampleSubmisson.csv")
sampleSubmission["Rating"] = np.clip(pred_rating,1,5)
sampleSubmission.to_csv("ML_hw5_submission.csv",index=None)

In [ ]:
def rmse(y_true, y_pred):
    y_pred = K.clip(y_pred, 1.0, 5.0)
    return K.sqrt(K.mean(K.pow(y_true - y_pred, 2)))
get_custom_objects().update({"rmse": rmse})
def get_model(num_users, num_items, mf_dim=10, layers=[64,32,16,8], reg_layers=[0,0,0,0], reg_mf=0):
    assert len(layers) == len(reg_layers)
    num_layer = len(layers) #Number of layers in the MLP
    # Input variables
    user_input = Input(shape=(1,), dtype='int32', name = 'user_input')
    item_input = Input(shape=(1,), dtype='int32', name = 'item_input')
    
    # Embedding layer
    MF_Embedding_User = Embedding(input_dim = num_users, output_dim = mf_dim, name = 'mf_embedding_user',
                                   embeddings_initializer = "random_normal", embeddings_regularizer= l2(reg_mf), input_length=1)
    MF_Embedding_Item = Embedding(input_dim = num_items, output_dim = mf_dim, name = 'mf_embedding_item',
                                   embeddings_initializer = "random_normal", embeddings_regularizer= l2(reg_mf), input_length=1)   

    MLP_Embedding_User = Embedding(input_dim = num_users, output_dim = int(layers[0]/2), name = "mlp_embedding_user",
                                   embeddings_initializer = "random_normal", embeddings_regularizer= l2(reg_layers[0]), input_length=1)
    MLP_Embedding_Item = Embedding(input_dim = num_items, output_dim = int(layers[0]/2), name = 'mlp_embedding_item',
                                   embeddings_initializer = "random_normal", embeddings_regularizer= l2(reg_layers[0]), input_length=1)   
    
    # MF part
    mf_user_latent = Flatten()(MF_Embedding_User(user_input))
    mf_item_latent = Flatten()(MF_Embedding_Item(item_input))
    mf_vector = multiply([mf_user_latent, mf_item_latent]) # element-wise multiply

    # MLP part 
    mlp_user_latent = Flatten()(MLP_Embedding_User(user_input))
    mlp_item_latent = Flatten()(MLP_Embedding_Item(item_input))
    mlp_vector = multiply([mlp_user_latent, mlp_item_latent])
    for idx in range(1, num_layer):
        layer = Dense(layers[idx], kernel_regularizer = l2(reg_layers[idx]), activation='relu', name="layer%d" %idx)
        mlp_vector = layer(mlp_vector)

    # Concatenate MF and MLP parts
    #mf_vector = Lambda(lambda x: x * alpha)(mf_vector)
    #mlp_vector = Lambda(lambda x : x * (1-alpha))(mlp_vector)
    predict_vector = concatenate([mf_vector, mlp_vector])
    
    # Final prediction layer
    prediction = Dense(1, activation='sigmoid', kernel_initializer='lecun_uniform', name = "prediction")(predict_vector)
    
    model = Model(inputs=[user_input, item_input], 
                  outputs=prediction)
    return model

In [ ]:
neuralCf = get_model(len(np.unique(train_user)), len(np.unique(train_movie)))

In [ ]:
hist = History()
# check_save  = ModelCheckpoint("models/model_"+str(i)+".h5",monitor='val_rmse',save_best_only=True)
early_stop = EarlyStopping(monitor="val_loss", patience=20)
neuralCf.compile(loss='mse', optimizer='adam', metrics=[rmse])
neuralCf.fit([train_user, train_movie], train_rating,
              batch_size=256,
              epochs=100,
              validation_split=0.05,
             callbacks=[hist, early_stop])

In [ ]:
neuralCf.summary()

In [19]:
np.save("user2id.npy",user2id)

In [22]:
file = open("user2id.pkl","wb")
pickle.dump(user2id, file)
file.close()

In [23]:
file = open("movie2id.pkl","wb")
pickle.dump(movie2id, file)
file.close()